In [2]:
!pip install transformers

In [3]:
import os

import random
import numpy as np

import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import (
    AdamW,
    Trainer,
    TrainingArguments,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    get_cosine_schedule_with_warmup,
)
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import gc

from transformers import pipeline
import re

In [4]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer()

# Load the training data
data = pd.read_csv('/kaggle/input/murderdata/Mysterious_Affair_at_Styles_Train_Data.csv')


In [5]:
data

,OrginalPage,Text
0,1,CHAPTER I.\nI GO TO STYLES\n\n\nThe intense in...
1,2,"looked his forty-five years. As a boy, though,..."
2,3,at the\ntime of their father's remarriage that...
3,4,"black beard,\nand wears patent leather boots i..."
4,5,"""I'm afraid you'll find it very quiet down her..."
...,...,...
202,203,hers! It concealed a very opposite emotion.\nT...
203,204,them to us. There was really more\nevidence ag...
204,205,something about the matter. He was nearly\ndri...
205,206,"""Yes, I quite understood your distress.""\n\n""P..."


In [6]:
!pip install nlpaug

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 2.0 MB/s eta 0:00:00a 0:00:01


In [7]:
# import nlpaug.augmenter.word as naw

# # Initialize the SynonymAug augmenter
# augmenter = naw.SynonymAug(aug_src='wordnet')

# # Augment each text sample in the list
# augmented_texts = []
# augmented_label = []
# for i,text in enumerate(data.Text):
#     augmented_text = augmenter.augment(text)
#     augmented_texts.append(augmented_text)
#     augmented_label.append(data.loc[i,"OriginalPage"])

In [8]:
# len(augmented_texts)

In [9]:
split_data = pd.DataFrame(columns=['text', 'OrginalPage'])

In [10]:
data

,OrginalPage,Text
0,1,CHAPTER I.\nI GO TO STYLES\n\n\nThe intense in...
1,2,"looked his forty-five years. As a boy, though,..."
2,3,at the\ntime of their father's remarriage that...
3,4,"black beard,\nand wears patent leather boots i..."
4,5,"""I'm afraid you'll find it very quiet down her..."
...,...,...
202,203,hers! It concealed a very opposite emotion.\nT...
203,204,them to us. There was really more\nevidence ag...
204,205,something about the matter. He was nearly\ndri...
205,206,"""Yes, I quite understood your distress.""\n\n""P..."


In [11]:
# for _, row in data.iterrows():
#     # Split the text into two equal parts
#     text1, text2 = row['Text'][:len(row['Text']) // 2], row['Text'][len(row['Text']) // 2:]
#     # Add the split texts to the dataframe as separate rows
#     split_data = split_data.append({'text': text1, 'OrginalPage': row['OrginalPage']}, ignore_index=True)
#     split_data = split_data.append({'text': text2, 'OrginalPage': row['OrginalPage']}, ignore_index=True)
    
# split_data=split_data.sample(frac=1)
# # Print the resulting dataframe
# split_data

In [12]:
def remove_newlines(text):
    return re.sub(r'\n', '', text)

In [13]:
# Preprocess the text data by lowercasing and removing punctuation
# split_data['text'] = split_data['text'].apply(remove_newlines)
data['Text'] = data['Text'].apply(remove_newlines)

In [14]:
# split_data

In [15]:
# X = split_data['text']
# X = X.str.lower().str.replace(r'[^\w\s]', '')
# y = split_data['OrginalPage']
X = data['Text']
X = X.str.lower().str.replace(r'[^\w\s]', '')
y = data['OrginalPage']

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [16]:
vectorizer = CountVectorizer()
tfidf_data = vectorizer.fit_transform(X).toarray()
features = vectorizer.get_feature_names_out()
tfidf_data = pd.DataFrame(tfidf_data, columns=features)

In [17]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest=train_test_split(tfidf_data, y, test_size=0.1, random_state=11)

In [18]:
!pip install lazypredict

In [19]:
from lazypredict.Supervised import LazyRegressor
from sklearn.utils import shuffle
import numpy as np

reg = LazyRegressor(verbose=0,ignore_warnings=False, custom_metric=None )
models,predictions = reg.fit(xtrain, xtest, ytrain, ytest)

 55%|█████▍    | 23/42 [01:52<01:16,  4.03s/it]

LassoLarsIC model failed to execute
You are using LassoLarsIC in the case where the number of samples is smaller than the number of features. In this setting, getting a good estimate for the variance of the noise is not possible. Provide an estimate of the noise variance in the constructor.


 79%|███████▊  | 33/42 [02:13<00:13,  1.46s/it]

RANSACRegressor model failed to execute
`min_samples` may not be larger than number of samples: n_samples = 186.


100%|██████████| 42/42 [02:34<00:00,  3.67s/it]


In [20]:
models

,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
Lars,148999692258756246747887811686301696.00,-73315298575921003357674571154717671424.00,478909726493936386048.00,1.61
SGDRegressor,411772189881522176.00,-202612506031202959360.00,796140061125.44,0.39
GaussianProcessRegressor,1.01,-2.13,98.94,0.55
KernelRidge,1.01,-2.10,98.43,0.45
OrthogonalMatchingPursuit,1.01,-1.94,95.89,0.56
DecisionTreeRegressor,1.01,-1.67,91.48,0.51
ExtraTreeRegressor,1.00,-1.42,87.03,0.47
KNeighborsRegressor,1.00,-1.18,82.50,0.36
MLPRegressor,1.00,-0.57,70.09,10.71


In [21]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import OrthogonalMatchingPursuitCV
from sklearn.linear_model import QuantileRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier , StackingRegressor, VotingRegressor
from sklearn.linear_model import OrthogonalMatchingPursuitCV, ElasticNetCV
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import GammaRegressor

model4=GammaRegressor()

model3 = HuberRegressor()
model2 = ElasticNetCV()
model1 = ElasticNet()
voting_model = StackingRegressor(estimators=[('model_1', model1), ('model_2', model2)
                                            , ('model_3', model3)])


# Make predictions on the test data

reg = voting_model.fit(tfidf_data, y)

In [22]:
# from sklearn.linear_model import OrthogonalMatchingPursuitCV
# reg = OrthogonalMatchingPursuitCV(cv=5).fit(tfidf_data, y)

In [23]:
# Load the test data
test_data = pd.read_csv('/kaggle/input/murderdata/Cain_s_Jawbone_Test_data.csv')
test_data['Text'] = test_data['Text'].apply(remove_newlines)
X_test = test_data['Text']
# Preprocess the test data in the same way as the training data
X_test = X_test.str.lower().str.replace(r'[^\w\s]', '')
# Extract features from the test data using the same vectorizer


In [24]:
testt = vectorizer.transform(X_test).toarray()
features = vectorizer.get_feature_names_out()
test_data = pd.DataFrame(testt, columns=features)

In [25]:
test_data.shape

(75, 9861)

In [26]:
y__pred=reg.predict(test_data)

In [27]:
y__pred

array([89.1062933 , 88.72177817, 97.10548396, 91.72477565, 89.60890915,
       87.78302445, 86.37876713, 89.15385853, 90.71660964, 92.61175649,
       92.92185335, 86.85221023, 86.75286672, 91.97861064, 90.56998329,
       88.66386666, 89.41424462, 90.08191534, 85.59093886, 88.4080165 ,
       85.59007961, 91.73808645, 91.45372134, 95.54226774, 89.98032521,
       86.43597276, 90.83868598, 90.75212571, 87.10541612, 87.37370866,
       83.07998824, 86.78725012, 96.70621387, 95.90989819, 85.89906264,
       89.66026749, 88.12096465, 89.74856748, 88.76918415, 91.22879266,
       85.63528302, 84.75925125, 86.15554633, 87.28026116, 85.41779856,
       91.72250281, 87.63784198, 92.37074363, 85.68737957, 89.70809503,
       88.42321809, 92.3480798 , 87.60807327, 89.30335072, 86.52051536,
       89.7950343 , 88.45126652, 89.00781714, 90.70485047, 86.3049473 ,
       88.25915312, 88.8680177 , 92.21228448, 87.62306764, 88.84057867,
       87.94706989, 95.91646225, 89.59775266, 86.6674816 , 88.52

In [28]:
pred=[round((i)) for i in y__pred]

In [29]:
predd=[int((i)) for i in y__pred]

In [30]:
len(set(pred)) , len(set(predd))

(12, 14)

In [31]:
set(pred), set(predd)

({83, 85, 86, 87, 88, 89, 90, 91, 92, 93, 96, 97},
 {83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 95, 96, 97})

In [56]:
pred

[89,
 89,
 97,
 92,
 90,
 88,
 86,
 89,
 91,
 93,
 93,
 87,
 87,
 92,
 91,
 89,
 89,
 90,
 86,
 88,
 86,
 92,
 91,
 96,
 90,
 86,
 91,
 91,
 87,
 87,
 83,
 87,
 97,
 96,
 86,
 90,
 88,
 90,
 89,
 91,
 86,
 85,
 86,
 87,
 85,
 92,
 88,
 92,
 86,
 90,
 88,
 92,
 88,
 89,
 87,
 90,
 88,
 89,
 91,
 86,
 88,
 89,
 92,
 88,
 89,
 88,
 96,
 90,
 87,
 89,
 89,
 88,
 93,
 96,
 92]

In [46]:
# eq=0
# diff=0
# equal=[]
# diffr=[]
# for i in range(75):
#     if pred[i]==predd[i]:
#         equal.extend([pred[i],i])
#         eq+=1
#     else:
#         diffr.extend([pred[i],predd[i],i])
#         diff+=1

In [47]:
eq, diff

(27, 48)

In [32]:
sub=pd.read_csv("/kaggle/input/murderdata/SampleSubmission.csv")

In [51]:
sub["CorrectPage"] = y__pred

In [52]:
sub.CorrectPage

0    89.11
1    88.72
2    97.11
3    91.72
4    89.61
      ... 
70   88.63
71   87.76
72   93.20
73   95.91
74   91.55
Name: CorrectPage, Length: 75, dtype: float64

In [53]:
sub.CorrectPage.nunique()

75

In [54]:
sub.to_csv("sub15.csv", index=False)

In [55]:
sub

,OriginalPage,CorrectPage
0,1,89.11
1,2,88.72
2,3,97.11
3,4,91.72
4,5,89.61
...,...,...
70,71,88.63
71,72,87.76
72,73,93.20
73,74,95.91
